### Параметры

In [ ]:
# === PARAMS ===
DATA_PATH = "/path/to/data"
TRAIN_FILE = "train.parquet"   # или .csv
TEST_FILE  = "test.parquet"

ID_COL      = "id"
TARGET_COL  = "target"         # для классификации/регрессии; если нет — None
DATE_COL    = None             # например "date" если есть время
NUM_COLS    = None             # список или None (авто-детект)
CAT_COLS    = None             # список или None (авто-детект)
MULTI_COLS  = []               # кат-колонки со списками/множествами (строка с разделителем)
LAT_COL = None        # например: "lat"
LON_COL = None        # например: "lon"
GEO_STEPS_M = [300, 1000]   # размеры «гридов» в метрах для агрегаций
NEIGHBOR_RADII_M = [300, 1000]  # радиусы соседей для плотностей

FAST = False                   # True => сэмплинг для быстрого прогона
SAMPLE_SIZE = 200_000          # если FAST=True, ограничение строк

# Настройки визуализации (опционально)
MAX_CATEG_UNIQUE_PREVIEW = 30
RARE_THRESHOLD = 0.01          # доля для пометки редких категорий
NEAR_CONST_STD_THRESH = 1e-6   # почти константные числовые

### Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

path = Path(DATA_PATH)
def read_any(p):
    return pd.read_parquet(p) if p.suffix==".parquet" else pd.read_csv(p)

train = read_any(path/TRAIN_FILE)
test  = read_any(path/TEST_FILE)

if FAST:
    train = train.sample(min(len(train), SAMPLE_SIZE), random_state=42)

# первичный обзор
display(train.head(3)); display(test.head(3))
print("train:", train.shape, "test:", test.shape)
print("mem train MB:", train.memory_usage(deep=True).sum()/1e6)
print("mem test  MB:", test.memory_usage(deep=True).sum()/1e6)

Смотреть: размеры, соответствие колонок, пример строк.
Реакция: если колонки не совпадают — зафиксировать список общих признаков; если память большая — сразу думать про downcast/categorical.

# СХЕМА И ЧИСТОТА

### Выявление типов, списки NUM/CAT

In [ ]:
all_cols = [c for c in train.columns if c != TARGET_COL]
if NUM_COLS is None:
    NUM_COLS = [c for c in all_cols if pd.api.types.is_numeric_dtype(train[c])]
if CAT_COLS is None:
    CAT_COLS = [c for c in all_cols if c not in NUM_COLS and c != DATE_COL]

print("ID:", ID_COL, "TARGET:", TARGET_COL, "DATE:", DATE_COL)
print("NUM:", len(NUM_COLS), "CAT:", len(CAT_COLS))

Смотреть: разумность списков, нет ли утечки (например, hash-колонка в NUM, которая по сути id).
Реакция: вынести технические id/ключи из фичей; явные даты — в DATE_COL.

### Пропуски, dtypes, константы

In [ ]:
# dtypes и пропуски
schema = pd.DataFrame({
    "dtype": train.dtypes.astype(str),
    "n_missing": train.isna().sum(),
    "missing_ratio": train.isna().mean()
}).sort_values("missing_ratio", ascending=False)
display(schema.head(20))

# почти константные числовые
near_const = []
for c in NUM_COLS:
    s = train[c].dropna()
    if s.size and s.std() < NEAR_CONST_STD_THRESH:
        near_const.append(c)
print("Near-constant numeric:", near_const[:20])

# полностью константные
const_cols = [c for c in all_cols if train[c].nunique(dropna=False)<=1]
print("Constant cols:", const_cols)

Смотреть: столбцы с высокой долей NaN, неподходящие dtypes (object вместо категорий), константы.
Реакция: наметить стратегию: дроп констант; для NaN >30–40% — либо продуманная импутация/таргет-aware, либо drop; object→category.

### Уникальность ID, дубликаты
Проверка уникальности ключа и дублей строк.

In [ ]:
if ID_COL in train:
    print("ID unique train:", train[ID_COL].is_unique)
if ID_COL in test:
    print("ID unique test:",  test[ID_COL].is_unique)

# дубликаты по всем колонкам (кроме TARGET)
dups = train.drop(columns=[TARGET_COL] if TARGET_COL in train else []).duplicated(keep=False).sum()
print("duplicate rows (w/o target):", dups)

# пересечения train/test по ID
if ID_COL in train and ID_COL in test:
    inter = np.intersect1d(train[ID_COL].values, test[ID_COL].values)
    print("ID overlap train/test:", len(inter))

Смотреть: неуникальные id, дубликаты, пересечения train/test.
Реакция: исправить ключ или собрать композитный; удалить дубли или агрегировать; пересечения train/test — флаг потенциальной утечки.

### Аномалии числовых (ноль/отрицательные/выбросы)

In [ ]:
num_profile = []
for c in NUM_COLS:
    s = pd.to_numeric(train[c], errors="coerce")
    q = s.quantile([.01,.05,.5,.95,.99])
    num_profile.append(dict(
        col=c, n_null=s.isna().sum(), zero=(s==0).sum(),
        neg=(s<0).sum(), p01=q.iloc[0], p05=q.iloc[1], p50=q.iloc[2],
        p95=q.iloc[3], p99=q.iloc[4], skew=s.skew()
    ))
pd.DataFrame(num_profile).sort_values("p99", ascending=False).head(20)


Смотреть: отрицательные там, где быть не должно; сильная асимметрия, хвосты.
Реакция: лог-преобразование (log1p), клиппинг по перцентилям (1–99%), биннинг.

### Дата/время (если есть)

In [ ]:
if DATE_COL:
    ts = pd.to_datetime(train[DATE_COL], errors="coerce")
    print("min/max:", ts.min(), ts.max())
    by_day = ts.dt.to_period("D").value_counts().sort_index()
    display(by_day.tail(10).to_frame("count"))

Смотреть: «дыры», сдвиги, хвост train vs test.
Реакция: планировать TimeSplit, эмбарго, временные фичи (сезон/неделя/скользящие).

# ЦЕЛЕВАЯ ПЕРЕМЕННАЯ

### Базовая статистика таргета

Распределение, выбросы, лог-масштаб (для регрессии); баланс классов (для классификации).

In [ ]:
import matplotlib.pyplot as plt

if TARGET_COL:
    y = train[TARGET_COL].dropna()
    display(y.describe(percentiles=[.01,.05,.5,.95,.99]))
    plt.figure(); y.hist(bins=50); plt.title("target"); plt.show()
    # лог-масштаб
    if (y>0).all():
        plt.figure(); np.log1p(y).hist(bins=50); plt.title("log1p(target)"); plt.show()
    # бинарный/мультикласс баланс
    if y.nunique()<=10 and pd.api.types.is_integer_dtype(y):
        display(y.value_counts(normalize=True).to_frame("ratio"))

Смотреть: скос, хвосты, нули/отрицательные, дисбаланс классов.
Реакция: для регрессии — лог-таргет или робаст-лоссы; для сильного дисбаланса — class_weight/стратификация.

### «Таргет во времени» (если DATE_COL)

Стабильность/дрейф.

In [ ]:
if TARGET_COL and DATE_COL:
    ts = pd.to_datetime(train[DATE_COL], errors="coerce")
    df = pd.DataFrame({TARGET_COL: train[TARGET_COL].values, "dt": ts})
    by_period = df.groupby(df["dt"].dt.to_period("W"))[TARGET_COL].agg(["count","mean","median"])
    display(by_period.tail(8))


### Бэйзлайны таргета

Простейшие ориентиpы.

In [ ]:
if TARGET_COL:
    if train[TARGET_COL].nunique() > 10:  # регрессия
        mae_mean = np.abs(train[TARGET_COL] - train[TARGET_COL].mean()).mean()
        print("MAE(mean baseline):", mae_mean)
    else:  # классификация
        maj = train[TARGET_COL].mode()[0]
        acc_maj = (train[TARGET_COL]==maj).mean()
        print("Majority class:", maj, "accuracy:", acc_maj)


Смотреть: насколько «тяжёл» таргет vs совсем тупой бейзлайн.
Реакция: понимать целевую дельту, которую надо превзойти.

### Подозрения на утечки

Быстрая эвристика: «слишком идеальные» корреляции, клоны таргета.

In [ ]:
if TARGET_COL:
    corrs = []
    for c in NUM_COLS[:200]:  # ограничим
        try:
            corrs.append((c, np.corrcoef(train[c].fillna(train[c].median()), train[TARGET_COL])[0,1]))
        except Exception:
            pass
    corr_df = pd.DataFrame(corrs, columns=["col","corr"]).sort_values("corr", ascending=False)
    display(corr_df.head(10))

Смотреть: подозрительно высокие |corr|, особенно для «технических» колонок.
Реакция: проверить происхождение фичи, возможно исключить/обработать от утечки.

# КАТЕГОРИАЛЬНЫЕ

### Кардинальность и пропуски по категориальным

Таблица «сколько уникальных», доля редких.

In [ ]:
cat_stats=[]
for c in CAT_COLS:
    vc = train[c].astype("object").value_counts(dropna=True)
    nuniq = vc.shape[0]
    rare_ratio = (vc[vc/train.shape[0] < RARE_THRESHOLD].sum())/train.shape[0] if nuniq else 0
    cat_stats.append({"col":c,"nunique":nuniq,"missing":train[c].isna().mean(),"rare_ratio":rare_ratio})
cat_stats = pd.DataFrame(cat_stats).sort_values("nunique", ascending=False)
display(cat_stats.head(20))


Смотреть: очень высокая кардинальность, большая доля редких, много NaN.
Реакция: high-card → планировать target/freq/hashing/CTR; rare→ объединение в __RARE__; продумать иммпутацию категориальных (в т.ч. явный __NONE__).

### Частоты топ-значений (просмотр)

Короткий превью для первых колонок.

In [ ]:
for c in CAT_COLS[:10]:
    print(f"\n{c} top values:")
    display(train[c].astype("object").value_counts(dropna=True).head(MAX_CATEG_UNIQUE_PREVIEW))


Смотреть: «мусорные» значения, разные написания одного и того же, спецсимволы.
Реакция: нормализовать (lower/strip), сопоставить с тестом.

### Совместные пары/тройки (кроссы)

Оценка «силы» комбинаций и потенциальных пересечений.

In [ ]:
from itertools import combinations
pairs = list(combinations(CAT_COLS[: min(8,len(CAT_COLS))], 2))
pair_stats=[]
for a,b in pairs:
    nun = train[a].astype("object").str.cat(train[b].astype("object"), sep="__").nunique()
    pair_stats.append({"pair":f"{a}×{b}", "nunique":nun})
display(pd.DataFrame(pair_stats).sort_values("nunique", ascending=False).head(15))

Смотреть: пары с огромной кардинальностью (может «взорвать» OHE), пары с небольшой — кандидаты для явных перекрёстных фич.
Реакция: для больших — hashing/TE; для компактных — можно OHE/CTR.

### «Таргет по категориям» (только осмотр, не для обучения)

Грубый намёк на информативность (не использовать агрегаты из этой ячейки в модели — только EDA!).

In [ ]:
if TARGET_COL and train[TARGET_COL].notna().any():
    previews=[]
    for c in CAT_COLS[:8]:
        g = train.groupby(c, dropna=True)[TARGET_COL].agg(["count","mean","median"]).sort_values("count", ascending=False).head(20)
        previews.append((c,g))
    for c,g in previews:
        print("\n", c); display(g.head(10))


Смотреть: категориальные уровни, сильно сдвигающие таргет; редкие, но «яркие» категории.
Реакция: кандидаты для TE/WOE (но в пайплайне — строго OOF), объединение редких, контроль утечек.

### unseen/only-in-test категории

Доля уровней, которых нет в train.

In [ ]:
unseen_report=[]
for c in CAT_COLS:
    tr = set(train[c].dropna().astype("object").unique())
    te = set(test[c].dropna().astype("object").unique())
    unseen = len(te - tr)
    unseen_report.append({"col":c,"only_in_test":unseen,"ratio_only_in_test": unseen/max(1,len(te))})
display(pd.DataFrame(unseen_report).sort_values("only_in_test", ascending=False).head(20))


Смотреть: много unseen → риск; особенно для ключевых фич.
Реакция: в пайплайне — маппинг __UNK__, freq/hashing/CTR вместо чистого OHE.

### MULTI_COLS (мультизначные категорики)

Быстрые агрегаты по множествам.

In [ ]:
import math

multi_stats=[]
for c in MULTI_COLS:
    # предполагаем строку с разделителем "," → правишь под свой формат
    lists = train[c].fillna("").astype(str).str.split(",")
    lens  = lists.map(len)
    multi_stats.append({
        "col": c, "mean_len": lens.mean(), "p95_len": lens.quantile(.95),
        "empty_ratio": (lens==0).mean()
    })
    # энтропия разметки (примерно)
    from collections import Counter
    cnt = Counter([tok for lst in lists for tok in lst if tok])
    tot = sum(cnt.values())
    probs = np.array([v/tot for v in cnt.values()])
    entropy = -(probs*np.log(probs+1e-12)).sum()
    print(c, "unique tokens:", len(cnt), "entropy:", round(entropy,2))
display(pd.DataFrame(multi_stats))


Смотреть: длины множеств, пустота, количество токенов — масштаб проблемы.
Реакция: multi-hot/частоты/TF-подобные агрегаты, avg rarity, top1 share, OOF-счётчики по токенам.

### Память и оптимизация типов

Прикинуть выгоду от приведения типов.

In [ ]:
def mem_mb(df): return df.memory_usage(deep=True).sum()/1e6
base = mem_mb(train)
opt = train.copy()

# downcast числовых
for c in NUM_COLS:
    if pd.api.types.is_float_dtype(opt[c]): opt[c] = pd.to_numeric(opt[c], downcast="float")
    if pd.api.types.is_integer_dtype(opt[c]): opt[c] = pd.to_numeric(opt[c], downcast="integer")

# категории
for c in CAT_COLS:
    opt[c] = opt[c].astype("category")

print("MB before:", round(base,1), "after:", round(mem_mb(opt),1))


Смотреть: экономия памяти → ускорение I/O/fit.
Реакция: применить в рабочем пайплайне (особенно на кластере с жёсткими лимитами).

### Числовые: профиль распределений и перекосы

Смотри: пропуски, нули/отрицательные, хвосты (p99/p01), сильный скос.
Реагируй: лог-преобразования (если >0), квантильный клиппинг, биннинг; проверить «аномальные» отрицательные.

In [ ]:
import numpy as np, pandas as pd

def profile_numeric(df, num_cols, pcts=(.01,.05,.5,.95,.99)):
    rows = []
    for c in num_cols:
        s = pd.to_numeric(df[c], errors="coerce")
        q = s.quantile(pcts)
        rows.append({
            "col": c,
            "missing_ratio": s.isna().mean(),
            "zero_ratio": (s==0).mean(),
            "neg_ratio": (s<0).mean(),
            "p01": q.iloc[0], "p05": q.iloc[1], "p50": q.iloc[2], "p95": q.iloc[3], "p99": q.iloc[4],
            "skew": s.skew(), "kurt": s.kurt()
        })
    out = pd.DataFrame(rows).sort_values("skew", ascending=False)
    print("\n[ЧИСЛОВЫЕ] Топ-15 по |скосу|:")
    print(out.reindex(out["skew"].abs().sort_values(ascending=False).index).head(15).to_string(index=False))
    return out

num_profile = profile_numeric(train, NUM_COLS)


### Числовые: выбросы и кандидаты на лог/клиппинг

Смотри: сколько значений попадает за перцентили 1/99 и 5/95.
Реагируй: лог1p (если все > −1), клип по квантилям, Huber/Quantile лоссы.

In [ ]:
def outlier_suggestions(prof):
    print("\n[ВЫБРОСЫ] Предварительные предложения:")
    for _, r in prof.iterrows():
        high_tail = (r["p99"] - r["p95"]) / (abs(r["p50"]) + 1e-9)
        low_tail  = (r["p05"] - r["p01"]) / (abs(r["p50"]) + 1e-9)
        log_ok = (r["p01"] > -0.9999)  # условно, чтобы log1p был безопасен
        tips = []
        if r["skew"] > 1 and log_ok:
            tips.append("log1p")
        if r["p99"] > r["p95"]*2 or r["p01"] < r["p05"]/2:
            tips.append("clip[1..99%]")
        if r["neg_ratio"] > 0 and r["p95"] > 0 and abs(r["skew"])>1:
            tips.append("robust_loss(Quantile/Huber)")
        if tips:
            print(f"  {r['col']}: " + ", ".join(tips))

outlier_suggestions(num_profile)

### Числовые: корреляции и мультиколлинеарность

Смотри: пары с |corr|>0.95; подозрительно дублирующие признаки.
Реагируй: удалить клоны/ко-лайнерные из пар, оставить один (или регуляризация).

In [ ]:
# Spearman — устойчивее к монотоничным нелинейностям
corr = train[NUM_COLS].corr(method="spearman")
pairs = []
for i, a in enumerate(NUM_COLS):
    for b in NUM_COLS[i+1:]:
        v = corr.loc[a,b]
        if pd.notna(v) and abs(v) >= 0.95:
            pairs.append((a,b,float(v)))
pairs = sorted(pairs, key=lambda x: -abs(x[2]))[:20]
print("\n[КОРРЕЛЯЦИИ] Топ пар |Spearman| ≥ 0.95:")
for a,b,v in pairs:
    print(f"  {a:>24s}  ~  {b:<24s}  corr={v:.3f}")

# VIF (опционально, если statsmodels установлен)
try:
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    import numpy as np
    X = train[NUM_COLS].select_dtypes(include=[np.number]).dropna().sample(min(50000, train.shape[0]), random_state=42)
    vifs = []
    for i, c in enumerate(X.columns):
        vifs.append((c, variance_inflation_factor(X.values, i)))
    print("\n[VIF] Топ-15 признаков с высокой мультиколлинеарностью:")
    for c, v in sorted(vifs, key=lambda x: -x[1])[:15]:
        print(f"  {c:>24s}  VIF={v:.2f}")
except Exception as e:
    print("\n[VIF] Пропущено (нет statsmodels или ошибка):", e)


### Мультизначные: глубже метрики редкости/энтропии

Смотри: средняя длина, доля пустых, число уникальных токенов, средняя «редкость», доля доминирующего токена.
Реагируй: OOF-счётчики частот/TE для токенов, avg_rarity, top1_share, ограничить словарь/хешировать.

In [ ]:
import numpy as np, pandas as pd
from collections import Counter

def multi_stats(df, cols, sep=","):
    print("\n[МУЛЬТИЗНАЧНЫЕ] Сводка:")
    for c in cols:
        lists = df[c].fillna("").astype(str).str.split(sep)
        lens  = lists.map(lambda x: len([t for t in x if t]))
        tokens = [t for lst in lists for t in lst if t]
        cnt = Counter(tokens); tot = sum(cnt.values()) or 1
        probs = np.array([v/tot for v in cnt.values()])
        # метрики
        mean_len = lens.mean(); p95_len = lens.quantile(.95)
        empty_ratio = (lens==0).mean()
        uniq = len(cnt)
        entropy = float(-(probs * np.log(probs+1e-12)).sum())
        top1_share = max(probs) if len(probs) else 0.0
        # средняя редкость токенов на строку
        inv_freq = {k: tot/v for k,v in cnt.items()}
        avg_rarity = lens.map(lambda L: np.mean([inv_freq.get(t,0) for t in L if t]) if L else 0).mean()
        print(f"  {c}: mean_len={mean_len:.2f}, p95_len={p95_len:.1f}, empty={empty_ratio:.2%}, "
              f"uniq={uniq}, entropy={entropy:.2f}, top1_share={top1_share:.2%}, avg_rarity={avg_rarity:.1f}")

multi_stats(train, MULTI_COLS)


### Время: покрытие, дыры, доля «свежих» данных

Смотри: min/max, пропуски дней/недель, доля последних 7/14 дней (для холдаута).
Реагируй: выбрать TimeSplit, подумать про эмбарго, сделать временные фичи/затухания.

In [ ]:
import pandas as pd, numpy as np

if DATE_COL:
    ts = pd.to_datetime(train[DATE_COL], errors="coerce")
    print("\n[ВРЕМЯ] Диапазон:", ts.min(), "→", ts.max())
    by_day = ts.dt.floor("D").value_counts().sort_index()
    total_days = (ts.max() - ts.min()).days + 1
    missing_days = total_days - by_day.shape[0]
    print(f"  дней в диапазоне: {total_days}, из них пустых: {missing_days}")
    # доля последних N дней
    for nd in [3,7,14,30]:
        cutoff = ts.max() - pd.Timedelta(days=nd-1)
        frac = (ts >= cutoff).mean()
        print(f"  доля записей за последние {nd:>2} дней: {frac:.2%}")
    # базовые признаки сезонности (по желанию)
    by_week = ts.dt.to_period("W").value_counts().sort_index()
    print("\n  хвост распределения по неделям:")
    print(by_week.tail(6).to_string())
else:
    print("\n[ВРЕМЯ] DATE_COL не задан — пропускаем блок.")


### Train/Test сравнение: числовые (сдвиги)

Смотри: относительные сдвиги mean/std, KS-тест; большие расхождения → риск дрейфа.
Реагируй: робастные трансформы, калибровки/клиппинг, адверсариалка, переоценка фич.

In [ ]:
import numpy as np, pandas as pd

try:
    from scipy.stats import ks_2samp
except Exception:
    ks_2samp = None

def num_drift_report(tr, te, num_cols):
    rows=[]
    for c in num_cols:
        a = pd.to_numeric(tr[c], errors="coerce").dropna()
        b = pd.to_numeric(te[c], errors="coerce").dropna()
        if len(a)==0 or len(b)==0:
            continue
        mean_a, std_a = a.mean(), a.std()
        mean_b, std_b = b.mean(), b.std()
        rel_mean = 0 if mean_a==0 else (mean_b-mean_a)/ (abs(mean_a)+1e-9)
        rel_std  = 0 if std_a==0  else (std_b-std_a) / (abs(std_a)+1e-9)
        ks_p = ks_2samp(a,b).pvalue if ks_2samp else np.nan
        rows.append({"col":c, "rel_mean":rel_mean, "rel_std":rel_std, "ks_p":ks_p})
    df = pd.DataFrame(rows)
    df["drift_score"] = df["rel_mean"].abs() + 0.5*df["rel_std"].abs() + (df["ks_p"].apply(lambda p: 1 if (not np.isnan(p) and p<1e-3) else 0))
    print("\n[DRIFT NUM] Топ-15 подозрительных по сдвигам mean/std/KS:")
    print(df.reindex(df["drift_score"].sort_values(ascending=False).index).head(15).to_string(index=False))
    return df

num_drift = num_drift_report(train, test, NUM_COLS)


### Train/Test сравнение: категориальные (дрейф, unseen)

Смотри: unseen-доли, JS-дивергенция топ-распределений.
Реагируй: __UNK__/__RARE__, freq/TE/hashing, ослабить зависимость модели от нестабильных фич.

In [ ]:
import numpy as np, pandas as pd

def js_divergence(p, q):
    p = np.asarray(p, dtype=float); q = np.asarray(q, dtype=float)
    p = p / (p.sum() + 1e-12); q = q / (q.sum() + 1e-12)
    m = 0.5*(p+q)
    def H(x): 
        x = np.where(x>0, x, 1)  # 0*log0=0
        return -(x*np.log2(x)).sum()
    return H(m) - 0.5*H(p) - 0.5*H(q)

def cat_drift_report(tr, te, cat_cols, topn=50):
    rows=[]
    for c in cat_cols:
        vtr = tr[c].astype("object").value_counts()
        vte = te[c].astype("object").value_counts()
        unseen = len(set(vte.index) - set(vtr.index))
        # JS по топ-N объединённому множеству
        keys = list(set(vtr.head(topn).index) | set(vte.head(topn).index))
        p = np.array([vtr.get(k,0) for k in keys], dtype=float)
        q = np.array([vte.get(k,0) for k in keys], dtype=float)
        js = js_divergence(p, q) if (p.sum()>0 and q.sum()>0) else np.nan
        rows.append({"col":c, "unseen_cnt":unseen, "js_top":js})
    df = pd.DataFrame(rows)
    df["drift_score"] = df["unseen_cnt"].rank(pct=True) + df["js_top"].fillna(0).rank(pct=True)
    print("\n[DRIFT CAT] Топ-15 по unseen и JS-дивергенции:")
    print(df.reindex(df["drift_score"].sort_values(ascending=False).index).head(15).to_string(index=False))
    return df

cat_drift = cat_drift_report(train, test, CAT_COLS)


### Адверсариальная валидация (быстрый снэпшот)

Смотри: AUC>0.7 — сильный дрейф/различимость train/test; какие фичи ведут к этому (importance).
Реагируй: переосмыслить эти фичи, добавить робастности/калибровок, корректнее валидацию.

In [ ]:
try:
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score

    # только числовые для скорости/простоты
    X_tr = train[NUM_COLS].copy()
    X_te = test[NUM_COLS].copy()
    X = pd.concat([X_tr, X_te], axis=0).fillna(0)
    y = np.array([0]*len(X_tr) + [1]*len(X_te))
    X = StandardScaler(with_mean=False).fit_transform(X)  # sparse-friendly если что

    Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    clf = LogisticRegression(max_iter=200, n_jobs=4)
    clf.fit(Xtr, ytr)
    auc = roc_auc_score(yva, clf.predict_proba(Xva)[:,1])
    print(f"\n[ADVERSARIAL] ROC-AUC (train vs test) на числовых: {auc:.3f}  (≥0.70 тревожно)")
except Exception as e:
    print("\n[ADVERSARIAL] Пропущено (нет sklearn или ошибка):", e)


### Рекомендации (авто-вывод по эвристикам)

Смотри: итоговую сводку — это «to-do» для фичеринга/валидации/постпроца.
Реагируй: перенеси пункты в свой рабочий план следующего ноутбука (features/train).

In [ ]:
print("\n=== РЕКОМЕНДАЦИИ (эвристики) ===")

# 1) числовые: лог/клип/робаст-лосс
skew_cols = num_profile[(num_profile["skew"]>1) & (num_profile["p01"]>-0.9999)]["col"].tolist()
if skew_cols:
    print("• Лог-преобразование кандидаты:", ", ".join(skew_cols[:10]), ("…+ ещё" if len(skew_cols)>10 else ""))

clip_cols = []
for _, r in num_profile.iterrows():
    if (r["p99"]>r["p95"]*2) or (r["p01"]<r["p05"]/2):
        clip_cols.append(r["col"])
if clip_cols:
    print("• Квантильный клиппинг (1–99%) для:", ", ".join(clip_cols[:10]), ("…+ ещё" if len(clip_cols)>10 else ""))

neg_cols = num_profile[num_profile["neg_ratio"]>0]["col"].tolist()
if neg_cols:
    print("• Отрицательные значения проверять (ошибки/семантика):", ", ".join(neg_cols[:10]), ("…+ ещё" if len(neg_cols)>10 else ""))

# 2) мультизначные
if MULTI_COLS:
    print("• Мультизначные: считать OOF-частоты токенов, avg_rarity, top1_share; ограничить словарь/хешировать.")

# 3) категориальные
hi_card = [c for c in CAT_COLS if train[c].nunique() > 1000]
if hi_card:
    print("• High-card категориальные → target/freq/WOE/CTR (строго OOF), hashing крестов:", ", ".join(hi_card[:10]))

# 4) корреляции/ко-лайнерность
if len(pairs) > 0:
    print("• Сильные линейные связи между числовыми (|ρ|≥0.95): оставить один из пары/регуляризовать.")

# 5) дрейф числовых
if not num_drift.empty and (num_drift["drift_score"].max() > 1.0):
    top_drift = num_drift.sort_values("drift_score", ascending=False).head(5)["col"].tolist()
    print("• Числовой дрейф в test → робастные трансформы/калибровки/адверсариалка. Топ:", ", ".join(top_drift))

# 6) дрейф категориальных
if not cat_drift.empty:
    w = cat_drift.sort_values("drift_score", ascending=False).head(5)
    print("• Категориальный дрейф (unseen/JS) → __UNK__/__RARE__, freq/TE/hashing. Топ:", ", ".join(w["col"].tolist()))

# 7) время
if DATE_COL:
    print("• Использовать TimeSplit; при сильной «хвостатости» — эмбарго и экспоненциальные затухания фич.")

# 8) валидация
print("• Валидация: без утечек (OOF для TE/CTR), проверка стабильности на альтернативном сплите.")
print("• Пост-проц: клиппинг/квантиль для регрессии, подбор порога τ для бинарной, инварианты top-k при ранжировании/мульти-лейбл.")


### GEO-1 — Валидация координат и базовая сводка

Смотри: NaN, выходы за диапазоны, (0,0), дубликаты точек, разброс по широте/долготе.
Реагируй: почистить/заполнить, проверить источники данных; слишком много (0,0) → явный баг.

In [ ]:
import numpy as np
import pandas as pd

if LAT_COL and LON_COL and LAT_COL in train and LON_COL in train:
    lat = pd.to_numeric(train[LAT_COL], errors="coerce")
    lon = pd.to_numeric(train[LON_COL], errors="coerce")

    n = len(train)
    bad_lat = ((lat<-90) | (lat>90) | lat.isna()).sum()
    bad_lon = ((lon<-180)| (lon>180)| lon.isna()).sum()
    zero_zero = ((lat.fillna(0)==0) & (lon.fillna(0)==0)).sum()

    print("\n[GEO] Валидация координат (train):")
    print(f"  всего: {n:,}")
    print(f"  некорректных lat: {bad_lat:,}  ({bad_lat/n:.2%})")
    print(f"  некорректных lon: {bad_lon:,}  ({bad_lon/n:.2%})")
    print(f"  ровно (0,0):      {zero_zero:,}  ({zero_zero/n:.2%})")

    lat_ok = lat.clip(-90, 90)
    lon_ok = lon.clip(-180, 180)
    print("\n[GEO] Диапазоны (после клипа к валидным границам):")
    print(f"  lat min/max: {lat_ok.min():.6f} .. {lat_ok.max():.6f}")
    print(f"  lon min/max: {lon_ok.min():.6f} .. {lon_ok.max():.6f}")

    # простая оценка дубликатов точек
    dup_points = pd.Series(list(zip(lat_ok.round(6), lon_ok.round(6)))).duplicated().sum()
    print(f"  дубликатов точек (до 1e-6 град): {dup_points:,}")
else:
    print("\n[GEO] LAT_COL/LON_COL не заданы или нет в данных — гео-блок пропущен.")


### «Грид» (агрегации по ячейкам) на 300м/1000м

Без внешних зависимостей: переводим метры в градусы (лат≈111.32км/°; lon зависит от cos(lat̄)).
Смотри: топ-плотности, размер «хвоста» (сколько ячеек с единичками).
Реагируй: фичи контрастов (среднее/медиана/плотности в радиусе), H3/квадробины в пайплайне.

In [ ]:
import numpy as np
import pandas as pd

def meters_to_deg_lat(m): return m/111_320.0
def meters_to_deg_lon(m, lat_deg):
    lat_rad = np.deg2rad(np.clip(lat_deg, -89.9, 89.9))
    return m/(111_320.0*np.cos(lat_rad))

if LAT_COL and LON_COL and LAT_COL in train and LON_COL in train:
    lat = pd.to_numeric(train[LAT_COL], errors="coerce")
    lon = pd.to_numeric(train[LON_COL], errors="coerce")
    lat_mean = float(lat.dropna().mean()) if lat.notna().any() else 0.0

    for step_m in GEO_STEPS_M:
        dlat = meters_to_deg_lat(step_m)
        dlon = meters_to_deg_lon(step_m, lat_mean)
        lat_bin = np.floor(lat/dlat).astype("Int64")
        lon_bin = np.floor(lon/dlon).astype("Int64")
        grid_id = (lat_bin.astype(str) + "_" + lon_bin.astype(str))

        vc = grid_id.value_counts(dropna=True)
        tot_cells = vc.shape[0]
        singletons = int((vc==1).sum())
        print(f"\n[GEO] Грид {step_m} м:")
        print(f"  всего ячеек: {tot_cells:,}, одиночных: {singletons:,}  ({singletons/max(1,len(grid_id)):.2%} записей-одиночек)")
        print("  топ-10 плотных ячеек (count):")
        print(vc.head(10).to_string())
else:
    print("\n[GEO] Пропуск «грид»-аналитики: нет координат.")


### GEO-3 — Плотность соседей в радиусах (BallTree, если есть sklearn)

Смотри: quantiles соседей при 300/1000м — это «плотность»; длинный «хвост» → разреженные регионы.
Реагируй: использовать эти плотности как фичи; для пустынных зон — бэкоф на глобальные/региональные агрегаты.

In [ ]:
try:
    from sklearn.neighbors import BallTree
    import numpy as np, pandas as pd

    if LAT_COL and LON_COL and LAT_COL in train and LON_COL in train:
        # haversine требует радианы
        lat_r = np.deg2rad(pd.to_numeric(train[LAT_COL], errors="coerce"))
        lon_r = np.deg2rad(pd.to_numeric(train[LON_COL], errors="coerce"))
        mask = lat_r.notna() & lon_r.notna()
        coords = np.c_[lat_r[mask], lon_r[mask]]
        if len(coords) == 0:
            print("\n[GEO] BallTree: нет валидных координат.")
        else:
            tree = BallTree(coords, metric="haversine")
            R = 6_371_000.0  # радиус Земли (м)
            print("\n[GEO] Плотность соседей (BallTree, haversine):")
            for r_m in NEIGHBOR_RADII_M:
                r_rad = r_m / R
                ind = tree.query_radius(coords, r=r_rad, count_only=False)
                # Кол-во соседей без себя:
                neigh_counts = np.array([len(ix)-1 for ix in ind], dtype=int)
                q = pd.Series(neigh_counts).quantile([.0,.25,.5,.75,.9,.99,1.0])
                print(f"  радиус {r_m:>4} м → quantiles соседей:")
                print(q.to_string())
    else:
        print("\n[GEO] Пропуск BallTree: нет LAT/LON.")
except Exception as e:
    print("\n[GEO] BallTree/ sklearn недоступен — блок пропущен.", e)


### GEO-4 — Центроиды/кластеры (быстрая оценка структуры)

Смотри: среднюю/макс дистанции до центроида; при больших значениях — «расползание».
Реагируй: фичи «расстояние до ближайшего кластера/центра»; региональные агрегаты.

In [ ]:
try:
    from sklearn.cluster import KMeans
    import numpy as np, pandas as pd

    if LAT_COL and LON_COL and LAT_COL in train and LON_COL in train:
        lat = pd.to_numeric(train[LAT_COL], errors="coerce")
        lon = pd.to_numeric(train[LON_COL], errors="coerce")
        m = lat.notna() & lon.notna()
        XY = np.c_[lat[m], lon[m]]
        n = len(XY)
        if n >= 1000:
            k = min(20, max(2, n//5000))  # грубая эвристика
            km = KMeans(n_clusters=k, random_state=42, n_init="auto").fit(XY)
            # евклидово в градусах (для грубой оценки)
            dist = ((XY - km.cluster_centers_[km.labels_])**2).sum(1)**0.5
            q = pd.Series(dist).quantile([.5,.9,.99])
            print("\n[GEO] Кластеры KMeans:")
            print(f"  k={k}, медиана/0.9/0.99 дистанции до центра (в градусах): {q.to_dict()}")
        else:
            print("\n[GEO] KMeans пропущен — слишком мало валидных точек (<1000).")
    else:
        print("\n[GEO] KMeans пропущен — нет координат.")
except Exception as e:
    print("\n[GEO] KMeans недоступен — блок пропущен.", e)


### GEO-5 — Train/Test покрытие по «гридам» и дрейф

Смотри: Jaccard пересечение ячеек, доля «ячеек только в test», топ-дрейф ячеек по долям.
Реагируй: делать back-off в постпроц/фичах для «новых» регионов (unseen bins), усилить глобальные/соседские агрегаты.

In [ ]:
import numpy as np, pandas as pd

def grid_bins(df, lat_col, lon_col, step_m):
    lat = pd.to_numeric(df[lat_col], errors="coerce")
    lon = pd.to_numeric(df[lon_col], errors="coerce")
    lat_mean = float(lat.dropna().mean()) if lat.notna().any() else 0.0
    dlat = step_m/111_320.0
    dlon = step_m/(111_320.0*np.cos(np.deg2rad(np.clip(lat_mean, -89.9, 89.9))))
    lat_bin = np.floor(lat/dlat).astype("Int64")
    lon_bin = np.floor(lon/dlon).astype("Int64")
    return (lat_bin.astype(str) + "_" + lon_bin.astype(str))

if LAT_COL and LON_COL and LAT_COL in train and LON_COL in train:
    for step_m in GEO_STEPS_M:
        gtr = grid_bins(train, LAT_COL, LON_COL, step_m)
        gte = grid_bins(test,  LAT_COL, LON_COL, step_m) if (LAT_COL in test and LON_COL in test) else pd.Series([],dtype=str)

        s_tr = set(gtr.dropna().unique())
        s_te = set(gte.dropna().unique())
        inter = len(s_tr & s_te)
        jacc = inter / max(1, len(s_tr | s_te))
        only_te = len(s_te - s_tr)

        print(f"\n[GEO] Train/Test по гриду {step_m} м:")
        print(f"  ячеек train: {len(s_tr):,}, test: {len(s_te):,}, пересечение: {inter:,}, Jaccard: {jacc:.3f}")
        print(f"  только в test: {only_te:,}  ({only_te/max(1,len(s_te)):.2%} от test-яч.)")

        # дрейф долей по топ-ячейкам
        vc_tr = gtr.value_counts(normalize=True)
        vc_te = gte.value_counts(normalize=True)
        keys = list((set(vc_tr.head(100).index) | set(vc_te.head(100).index)))
        drift = []
        for k in keys:
            drift.append((k, float(vc_tr.get(k,0)-vc_te.get(k,0))))
        drift = sorted(drift, key=lambda x: -abs(x[1]))[:10]
        print("  топ-10 ячеек по |разнице долей train-test|:")
        for k, d in drift:
            print(f"    {k}: Δ={d:+.4f}")
else:
    print("\n[GEO] Нет координат — сравнение покрытий пропущено.")


### GEO-6 — Таргет vs гео (если есть TARGET_COL)

Смотри: насколько таргет «гуляет» по ячейкам — это кандидат для локальных агрегатов/калибровок.
Реагируй: добавить контрасты: mean/median/quantiles таргета по гриду (OOF!), расстояние до «дорогих/дешёвых» зон.

In [ ]:
import numpy as np, pandas as pd

if TARGET_COL and LAT_COL and LON_COL and (TARGET_COL in train):
    y = pd.to_numeric(train[TARGET_COL], errors="coerce")
    for step_m in GEO_STEPS_M:
        g = grid_bins(train, LAT_COL, LON_COL, step_m)
        df = pd.DataFrame({"grid": g, "y": y}).dropna()
        if df.empty:
            print(f"\n[GEO] Таргет по гриду {step_m} м: нет валидных точек.")
            continue
        agg = df.groupby("grid")["y"].agg(["count","mean","median","std"]).sort_values("count", ascending=False)
        print(f"\n[GEO] Таргет по гриду {step_m} м (топ-10 по покрытию):")
        print(agg.head(10).to_string())
        print("  разброс mean по ячейкам (q05/median/q95):",
              agg["mean"].quantile(.05), agg["mean"].quantile(.5), agg["mean"].quantile(.95))
else:
    print("\n[GEO] Нет TARGET или координат — анализ таргета по гео пропущен.")


### GEO-7 — Резюме рекомендаций по гео

Смотри: краткий список действий, завязанный на факты выше.
Реагируй: перенеси пункты в следующий ноут (features).

In [ ]:
print("\n=== РЕКОМЕНДАЦИИ ПО ГЕО ===")

# 1) Валидность
if LAT_COL and LON_COL and LAT_COL in train and LON_COL in train:
    lat = pd.to_numeric(train[LAT_COL], errors="coerce")
    lon = pd.to_numeric(train[LON_COL], errors="coerce")
    if ((lat.fillna(0)==0) & (lon.fillna(0)==0)).mean() > 0.001:
        print("• Есть (0,0) точки → почистить/маскировать, иначе испортят локальные агрегаты.")

# 2) Гриды/контрасты
print("• Добавить локальные контрасты по гриду (например,", ", ".join([f"{m}м" for m in GEO_STEPS_M]), ") — count/mean/median/quantiles таргета (строго OOF).")

# 3) Плотности/соседи
print("• Фичи плотности: число соседей в радиусах", NEIGHBOR_RADII_M, "метров; доля одиночек; расстояние до ближайшего центра/кластера.")

# 4) Train/Test покрытие
print("• Есть unseen/only-in-test ячейки? → бэкоф на глобальные/региональные агрегаты; избегать жёсткой привязки к редким ячейкам.")

# 5) Постпроц/устойчивость
print("• Если сильный гео-дрейф таргета: клиппинг по регионам/локальные калибровки; TimeSplit с эмбарго, если есть время.")
